In [2]:
import geopandas as gpd
import laspy
import pdal
import json,os,subprocess,sys
import pandas as pd
import requests
from dateutil.parser import parse
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import numpy as np
import math
import pyproj    
import shapely
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from functools import partial



# import urllib library
from urllib.request import urlopen
sys.path.append(os.path.abspath(os.path.join('../')))
from geolidar import state_mapper
from geolidar.mapper import state_mapper_variables
from rasterio.plot import show
import rasterio
from rasterio.plot import show_hist

ModuleNotFoundError: No module named 'geopandas'

In [3]:
!pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-0.11.0-py3-none-any.whl (1.0 MB)
  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
  Using cached Fiona-1.8.20.tar.gz (1.3 MB)
  Using cached Fiona-1.8.19.tar.gz (1.3 MB)
  Using cached Fiona-1.8.18.tar.gz (1.3 MB)
  Using cached Fiona-1.8.17.tar.gz (1.3 MB)
  Using cached Fiona-1.8.16.tar.gz (1.3 MB)
  Using cached Fiona-1.8.15.tar.gz (1.3 MB)
  Using cached Fiona-1.8.14.tar.gz (1.3 MB)
  Using cached Fiona-1.8.13.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.13.tar.gz (1.2 MB)
  Using cached Fiona-1.8.12.tar.gz (1.2 MB)
  Using cached Fiona-1.8.11.tar.gz (1.2 MB)
  Using cached Fiona-1.8.10.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post2.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.post1.tar.gz (1.2 MB)
  Using cached Fiona-1.8.9.tar.gz (1.2 MB)
  Using cached Fiona-1.8.8.tar.gz (1.7 MB)
  Using cached Fiona-1.8.7.tar.gz (1.7 MB)
  Using cached Fiona-1.8.6.tar.gz (1.7 MB)
  Using cached F

    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-9qpe381x\\fiona_1f385a5e445b49d78d7a8ee1ae652694\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-9qpe381x\\fiona_1f385a5e445b49d78d7a8ee1ae652694\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\user\AppData\Local\Temp\pip-pip-egg-info-91vcgzyy'
         cwd: C:\Users\user\AppData\Local\Temp\pip-install-9qpe381x\fiona_1f385a5e445b49d78d7a8ee1ae652694\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment vari

  Using cached geopandas-0.10.0-py2.py3-none-any.whl (1.0 MB)
  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached geopandas-0.8.2-py2.py3-none-any.whl (962 kB)
  Using cached Fiona-1.7.13.tar.gz (731 kB)
  Running setup.py clean for fiona
Failed to build fiona
    Running setup.py install for fiona: started
    Running setup.py install for fiona: finished with status 'error'



    ERROR: Command errored out with exit status 1:
     command: 'C:\ProgramData\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-9qpe381x\\fiona_e8cb7aec2fb54ee7974b0617653cd007\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-9qpe381x\\fiona_e8cb7aec2fb54ee7974b0617653cd007\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\user\AppData\Local\Temp\pip-pip-egg-info-tov1a10k'
         cwd: C:\Users\user\AppData\Local\Temp\pip-install-9qpe381x\fiona_e8cb7aec2fb54ee7974b0617653cd007\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment var

In [ ]:
from vi_support import *

In [ ]:
_,state_list_,_=run_command('aws s3 ls --no-sign-request s3://usgs-lidar-public/',print_constantly=False)
state_list = [eval(i) for i in state_list_]
state_list = [i.decode('utf-8') for i in state_list]
state_list = [i.lstrip() for i in state_list]
state_list = [i.replace('/\n','') for i in state_list]
state_list = [i.replace('PRE ','') for i in state_list]

In [ ]:
with open('../data/metadata.csv','w') as f:
    f.write('state_mapper={')
    for item in state_list:
        f.write(f"'{item}':'{item}',\n")
    f.write('}')